In [1]:
# %load ../uoicorr_imports.py
import h5py

import numpy as np
import matplotlib.pyplot as plt

import sys, os, pdb

# Hack to import pyuoi
parent_path, current_dir = os.path.split(os.path.abspath('.'))
while current_dir not in ['nse']:
    parent_path, current_dir = os.path.split(parent_path)
p = os.path.join(parent_path, current_dir)
# Add analysis
if p not in sys.path:
    sys.path.append(p)

import subprocess
hname = subprocess.check_output('hostname')

if 'ankitnse'.encode() in hname:

    if '%s/uoicorr' % p not in sys.path:
        sys.path.append('%s/uoicorr' % p)
    if '%s/PyUoI' % p not in sys.path:
        sys.path.append('%s/PyUoI' % p)

else:
    # And standard list of subdirectories
    if '%s\\pyuoi' % p not in sys.path:
        sys.path.append('%s\\pyuoi' % p)
    if '%s\\uoicorr' % p not in sys.path:
        sys.path.append('%s\\uoicorr' % p)

from postprocess import postprocess_file, postprocess_dir
from utils import FNR, FPR, selection_accuracy

from pyuoi.linear_model.gtv import GraphTotalVariance

In [2]:
# Generate data to test the GTV. Use a block correlation structure and a small number of features
from utils import gen_covariance, gen_beta, gen_data
from sklearn.linear_model import Lasso

In [3]:
cov = gen_covariance('block', n_features = 30, block_size = 5, correlation=0.5)
beta = gen_beta(n_features = 30, block_size = 5, sparsity = 0.5)
X, X_test, y, y_test = gen_data(n_samples = 100, n_features = 30, kappa = 0.3, covariance = cov, beta = beta)

In [4]:
l = Lasso(alpha = 1)

In [5]:
l.fit(X, y)
l.coef_

array([ 1.57498944,  0.5033593 ,  0.        ,  6.81150028, -0.        ,
        0.        ,  2.15121536,  0.        ,  0.12376779, -0.        ,
        1.297592  , -0.        ,  5.11566914,  0.        ,  1.47180866,
        7.19079398,  0.        ,  2.51409457, -0.        , -0.        ,
        5.96498834,  7.29847832,  0.        , -1.06631919,  0.        ,
        0.44189012,  7.4948836 ,  7.98781847,  1.2082049 ,  0.        ])

In [6]:
from sklearn.metrics import r2_score

In [10]:
# Test on a 3D grid of tuning parameters
lambda_S = np.linspace(0, 1, num = 10)
lambda_TV = np.linspace(0, 1, num = 10)
lambda_1 = np.linspace(0, 1, num = 10)

r2_scores = np.zeros((10, 10, 10))
sa = np.zeros((10, 10, 10))

for i1, l1 in enumerate(lambda_S):
    for i2, l2 in enumerate(lambda_TV):
        for i3, l3 in enumerate(lambda_1):
            gtv = GraphTotalVariance(l1, l2, l3)
            gtv.fit(X, y, cov)
            gtv.coef_[gtv.coef_ < 1e-6] = 0
            r2_scores[i1, i2, i3] = r2_score(y_test, np.dot(X_test, gtv.coef_))
            sa[i1, i2, i3] = selection_accuracy(beta.ravel(), gtv.coef_)

In [11]:
sa

array([[[0.86666667, 0.86666667, 0.86666667, 0.88333333, 0.9       ,
         0.9       , 0.9       , 0.9       , 0.9       , 0.9       ],
        [0.86666667, 0.85      , 0.85      , 0.86666667, 0.85      ,
         0.85      , 0.85      , 0.85      , 0.83333333, 0.83333333],
        [0.86666667, 0.85      , 0.83333333, 0.85      , 0.83333333,
         0.81666667, 0.8       , 0.8       , 0.75      , 0.75      ],
        [0.86666667, 0.85      , 0.86666667, 0.86666667, 0.78333333,
         0.78333333, 0.73333333, 0.71666667, 0.7       , 0.7       ],
        [0.86666667, 0.86666667, 0.85      , 0.8       , 0.76666667,
         0.7       , 0.7       , 0.7       , 0.7       , 0.7       ],
        [0.86666667, 0.85      , 0.8       , 0.75      , 0.7       ,
         0.7       , 0.7       , 0.7       , 0.7       , 0.7       ],
        [0.86666667, 0.85      , 0.78333333, 0.73333333, 0.7       ,
         0.7       , 0.7       , 0.7       , 0.7       , 0.7       ],
        [0.86666667, 0.85  

In [12]:
r2_scores

array([[[0.65902751, 0.68357176, 0.69846844, 0.70686526, 0.7110454 ,
         0.71433328, 0.71779063, 0.7204957 , 0.72293615, 0.72518635],
        [0.65902753, 0.69146382, 0.70933766, 0.71683342, 0.72142354,
         0.72434071, 0.72643904, 0.72717108, 0.72658055, 0.72435026],
        [0.65902761, 0.69775886, 0.71570553, 0.72139091, 0.7240564 ,
         0.72484268, 0.72201103, 0.71673171, 0.70891776, 0.69800178],
        [0.65902774, 0.70320194, 0.7186961 , 0.72198206, 0.7223658 ,
         0.71567105, 0.7024495 , 0.68586672, 0.67174537, 0.65499195],
        [0.65902792, 0.70798726, 0.72058868, 0.72140316, 0.71295471,
         0.69283978, 0.67361944, 0.65118873, 0.63015851, 0.61562185],
        [0.65902815, 0.71194092, 0.72001752, 0.71567307, 0.69294273,
         0.66846405, 0.63878832, 0.61803895, 0.60554447, 0.60070614],
        [0.65902843, 0.71469473, 0.71945687, 0.70358478, 0.6735145 ,
         0.63799541, 0.61465053, 0.60285977, 0.59636518, 0.59499761],
        [0.65902877, 0.7163